### Dyer-Lashof-Cohen basis

In [ ]:
from itertools import combinations, combinations_with_replacement, permutations, combinations, product, chain, tee
from sympy.utilities.iterables import multiset_permutations
from clesto import *

def symmetric_orbit(surj):
    translation = {(1,2): (1,2,3), (1,3): (1,3,2), (2,1): (2,1,3),
                   (2,3): (3,1,2), (3,1): (2,3,1), (3,2): (3,2,1)}
    rep = Surjection_element(torsion=3)
    for k, v in surj.items():
        perm = SymmetricModule_element({translation[k[:2]]: 1}, torsion=surj.torsion)
        new = perm * Surjection_element({k:v}, torsion=surj.torsion)
        rep += new
    return rep

def sign_symmetric_orbit(surj):
    translation = {(1,2): (1,2,3), (1,3): (1,3,2), (2,1): (2,1,3),
                   (2,3): (3,1,2), (3,1): (2,3,1), (3,2): (3,2,1)}
    sign = {(1,2): -1, (1,3): -1, (2,1): -1,
            (2,3): 1, (3,1): +1, (3,2): -1}
    rep = Surjection_element(torsion=3)
    for k, v in surj.items():
        perm = SymmetricModule_element({translation[k[:2]]: sign[k[:2]]}, torsion=surj.torsion)
        new = perm * Surjection_element({k:v}, torsion=surj.torsion)
        rep += new
    return rep

def sign_permutation(permutation):
    a = [(i + 1, j) for i, j in enumerate(permutation) if i+1 != j]
    b = [(i, j) for i, j in a if permutation[j-1] == i]
    if (len(a) - int(len(b)/2))%2:
        return -1
    else:
        return 1

def symmetric_orbit_barratt_eccles(be, sign_representation=False):
    rep = BarrattEccles_element(torsion=be.torsion)
    for k, v in be.items():
        inverse = tuple(k[0].index(i+1)+1 for i in range(len(k[0])))
        perm = SymmetricModule_element({inverse: 1}, torsion=be.torsion)
        new = perm * BarrattEccles_element({k: v}, torsion=be.torsion)
        if sign_representation:
            new = sign_permutation(k[0]) * new
        rep += new
    return rep

def extra_reduce(surj, r=3):
    return Surjection_element({k: v for k, v in surj.items() if set(k) == set(range(1, r+1))})

def cyclic_orbit(surj):
    pass

def get_symmetric_basis():
    basis = set()
    for x in combinations_with_replacement((1,2,3), 5):
        for y in permutations(x):
            if set(y) == {1,2,3} and Surjection_element({y:1}):
                basis.update(symmetric_orbit(Surjection_element({y:1}, torsion=3)))
    return basis

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def get_basis(r, n, torsion='free'):
    basis = []
    surjections = (chain.from_iterable(multiset_permutations(y) 
        for y in combinations_with_replacement(range(1, r+1), r+n)))
    for s in surjections:
        if all([i != j for i, j in pairwise(s)]) and set(s) == set(range(1, r+1)):
#             yield Surjection_element({tuple(s):1}, torsion=torsion)
            yield tuple(s)
        
def phi(i, j, permutations):
    '''the restriction to a pair of integers a la Berger, i.e pull back to the arity 2 part'''
    def _phi(i, j, permutation):
        try:
            a, b = permutation.index(i), permutation.index(j)

        except ValueError:
            print(i, j, permutation)

        if a < b:
            return (1, 2)
        else:
            return (2, 1)
        
    return tuple(_phi(i, j, p) for p in permutations)

In [ ]:
r = 3 # arity
n = 2 # dimension

basis_S = set(permutations(range(1, r+1), r))
x = combinations_with_replacement(basis_S, n+1)
simplices = set()
for y in x:
    simplices |= set(permutations(y))

# removes degenerate simplices
simplices = tuple(spx for spx in simplices 
                  if all(spx[i] != spx[i + 1] for i in range(len(spx) - 1)))

cell = []
for spx in simplices:
    lower = []
    last = []
    for i, j in combinations(range(1, r+1), 2):
        a = phi(i, j, spx)
        degenerate = not bool(BarrattEccles_element({a: 1}))
        lower.append(degenerate)
        last.append((a[-1] == (1, 2)))
    
    if any(lower):
        print(spx, lower, last)
        print(BarrattEccles_element({spx:1}).table_reduction())
#     if all(include):
#         cell.append(spx)
#         print(spx)

# print(cell)
# be = BarrattEccles_element({((1, 2, 3), (3, 2, 1), (1, 2, 3)):1}, torsion=3)
# print(be.table_reduction())



In [ ]:
r = 3 # arity
n = 2 # dimension

basis_S = set(permutations(range(1, r+1), r))
x = combinations_with_replacement(basis_S, n)
pre_simplices = set()
for y in x:
    pre_simplices |= set(permutations(y))
pre_simplices = tuple(z + (tuple(range(1, r+1)),) for z in pre_simplices)

# removes degenerate simplices
simplices = tuple(spx for spx in pre_simplices 
                  if all(spx[i] != spx[i + 1] for i in range(len(spx) - 1)))
cell = []
for spx in simplices:
    top = []
    for i, j in combinations(range(1, r+1), 2):
        top.append(bool(BarrattEccles_element({phi(i, j, spx) : 1})))
    #print(spx, top)
    if not any(top):
        cell.append(spx)
#         print(spx)

print(cell)
# be = BarrattEccles_element({((1, 2, 3), (3, 2, 1), (1, 2, 3)):1}, torsion=3)
# print(be.table_reduction())


